<style>
.myDiv {
  border: 5px outset red;
  background-color: black;
  color: white;
  text-align: center;
}
</style>

<div class="myDiv">
  <p>This notebook is a work in progress. Trying to cover most common NLP techniques.</p>
</div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub
from kaggle_datasets import KaggleDatasets

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

import transformers
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from tokenizers import BertWordPieceTokenizer
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from keras import Input, Model
from keras.optimizers import Adam

print(tf.version.VERSION)

Files<br><br>
jigsaw-toxic-comment-train.csv - data from our first competition. The dataset is made up of English comments from Wikipedia’s talk page edits.<br>
jigsaw-unintended-bias-train.csv - data from our second competition. This is an expanded version of the Civil Comments dataset with a range of additional labels.<br>
sample_submission.csv - a sample submission file in the correct format<br>
test.csv - comments from Wikipedia talk pages in different non-English languages.<br>
validation.csv - comments from Wikipedia talk pages in different non-English languages.<br>
jigsaw-toxic-comment-train-processed-seqlen128.csv - training data preprocessed for BERT<br>
jigsaw-unintended-bias-train-processed-seqlen128.csv - training data preprocessed for BERT<br>
validation-processed-seqlen128.csv - validation data preprocessed for BERT<br>
test-processed-seqlen128.csv - test data preprocessed for BERT<br><br>
Columns<br><br>
id - identifier within each file.<br>
comment_text - the text of the comment to be classified.<br>
lang - the language of the comment.<br>
toxic - whether or not the comment is classified as toxic. (Does not exist in test.csv.)<br>

In [ ]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code

try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

In [ ]:
#toxic_comment_train = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
toxic_comment_train_processed = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv')

#unintended_bias_train = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv')
unintended_bias_train_processed = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv')

#validation = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
validation_processed = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv')


#test = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/test.csv')
#test_processed = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv')


'''
> test-processed-seqlen128.csv is missing the lang column, so we load test.csv too
> All other columns are retained in the toxic comment train and unintended bias train datasets, so we need not load unprocessed.
> All processed datasets have common new column information namely, 'input_word_ids', 'input_mask', 'all_segment_id'

''' 
sample_submission = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

'''
Submission Format being...
	id	toxic
0	0	0.5
1	1	0.5
2	2	0.5
3	3	0.5
4	4	0.5
'''
processed_column_names = ['input_word_ids','input_mask','all_segment_id']
cmn_toxic_params = ['toxic', 'severe_toxic','obscene','threat','insult','identity_hate']

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
#nltk.download('stopwords')

def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.sub(r'[0-9]+' , '' ,text)
    text = re.sub(r'\s([@][\w_-]+)', '', text).strip()
    text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'https?://\S+|www\.\S+','',text) #hyperlinks
    text = re.sub(r'[!@#$|."]','',text) #symbols
    text = text.replace("#" , " ")
    encoded_string = text.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string

sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

def without_stopwords(comment):
    comment_tokens_with_sw = word_tokenize(comment)
    tokens_without_sw = [word for word in comment_tokens_with_sw if not word in all_stopwords]
    filtered_sentence = (" ").join(tokens_without_sw)
    return filtered_sentence

def plot_accuracy_and_loss_from_history(history):
    color = sns.color_palette()
    acc = history.history['accuracy']
    loss = history.history['loss']
    epochs = range(len(acc))
    sns.lineplot(epochs, acc, label='Training Accuracy')
    sns.lineplot(epochs, loss,label='Training Loss')
    plt.title('Training Accuracy and loss')
    plt.legend()
    plt.figure()
    plt.show()
    
def plot_roc_curve_from_prediction(fpr,tpr,roc_auc):
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
    
def do_history_stuff(history):
    plot_accuracy_and_loss_from_history(history)
    
def roc_auc(predictions,target):
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    plot_roc_curve_from_prediction(fpr,tpr,roc_auc)
    return roc_auc

def convert_continious_to_discrete_labels(ubtp, labels):
    
    for i in labels:
        ubtp[i] = ubtp[i].apply(lambda x: 1 if x>0 else 0)
        
    return ubtp

def print_neat_bad_comments(unintended_bias_train_processed, extra_labels):
    print('Percentage of missing values : {}'.format((unintended_bias_train_processed.isna().sum().max()/unintended_bias_train_processed.shape[0])*100))
    value_counts = dict(unintended_bias_train_processed.isna().sum())
    columns_with_nans = [column_name  for column_name, value in zip(value_counts.keys(), value_counts.values()) if (value > 0)]
    print(columns_with_nans)
    print(unintended_bias_train_processed.columns)
    '''
    'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow','sad', 'likes', 'disagree', 'sexual_explicit',
    'identity_annotator_count', 'toxicity_annotator_count',
    '''
    ubtp = convert_continious_to_discrete_labels(unintended_bias_train_processed,['toxic','obscene','identity_hate','insult','threat'])
    
    ubtp_original_labels = [col for col in ubtp.columns if not col in extra_labels]
    print(ubtp_original_labels)

    ubtp['negative_comment_count'] = ubtp['toxic']+ubtp['severe_toxic']+ubtp['obscene']+ubtp['identity_hate']+ubtp['insult']+ubtp['threat']
    neat_comments =  ubtp.loc[ubtp['negative_comment_count'] == 0.0,:]
    bad_comments = ubtp.loc[ubtp['negative_comment_count'] != 0,:]

    print('Total number of neat comments : {}'.format(neat_comments.shape[0]))
    print('Total number of bad comments : {}'.format(bad_comments.shape[0]))

    print('Total number of missing extra labels in neat comments : {}'.format(neat_comments.isna().sum().max()))
    print('Total number of missing parent_id in  neat comments : {}'.format(neat_comments.isna().sum().parent_id))

    print('Total number of missing extra labels in bad comments : {}'.format(bad_comments.isna().sum().max()))
    print('Total number of missing parent_id in  bad comments : {}'.format(bad_comments.isna().sum().parent_id))

    print('Number of comments raising a red flag : {}'.format(ubtp.shape[0] - neat_comments.shape[0]))
    
def pie_chart_combined_negative_category(toxic_comment_train_processed,cmn_toxic_params):
    toxic_comment_category_value_counts = toxic_comment_train_processed[cmn_toxic_params+['id']].groupby(by=cmn_toxic_params).count().sort_values('id',ascending=False)
    toxic_combination_matrix = pd.DataFrame(toxic_comment_category_value_counts.iloc[1:,0])
    toxic_combination_matrix = toxic_combination_matrix.reset_index()
    tcm = toxic_combination_matrix[cmn_toxic_params].astype('str')
    tcm['comb_str'] = ''
    toxic_combination_matrix['comb_str'] = tcm['toxic'] + tcm['severe_toxic'] + tcm['obscene'] + tcm['threat'] + tcm['insult'] + tcm['identity_hate']

    toxic_combination_matrix['percentage'] = toxic_combination_matrix['id']/toxic_combination_matrix['id'].sum()
    fig1, ax1 = plt.subplots(figsize=(10,10))
    explode = (0.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    ax1.pie(toxic_combination_matrix['percentage'], explode=explode, labels=toxic_combination_matrix['comb_str'],shadow=True, startangle=20)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
    plt.show()
    return toxic_combination_matrix
    
def toxic_weight_comparision(toxic_combination_matrix, cmn_toxic_params):
    '''
    I want to allot weights to the severity of the comments

    insult = 1
    obscene	= 2 
    toxic = 3
    severe_toxic = 4
    identity_hate = 5
    threat = 6
    '''
    nmp = toxic_combination_matrix[cmn_toxic_params].astype('int')
    toxic_combination_matrix['weight'] = nmp['toxic']*3 + nmp['severe_toxic']*4 + nmp['obscene']*2 + nmp['threat']*6 + nmp['insult']*1 + nmp['identity_hate']*5
    toxic_combination_matrix['id'] = toxic_combination_matrix['id'].astype('int')

    intensity = toxic_combination_matrix.loc[:,['id','weight']].groupby(by='weight').sum()
    plt.figure(figsize=(20,5))
    plt.bar(x=intensity.index.astype('str'),height=intensity['id'])

### Toxic Comment train processed

In [ ]:
print(toxic_comment_train_processed.shape) 
toxic_comment_train_processed.head()

### Unintended Bias train processed

In [ ]:
print(unintended_bias_train_processed.shape) 
unintended_bias_train_processed.rename(columns={'severe_toxicity': 'severe_toxic', 'identity_attack':'identity_hate'},inplace=True) 
unintended_bias_train_processed.head()

In [ ]:
extra_labels = ['asian', 'atheist', 'bisexual','black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu','homosexual_gay_or_lesbian', 'intellectual_or_learning_disability','jewish', 'latino', 'male', 'muslim', 'other_disability','other_gender', 'other_race_or_ethnicity', 'other_religion','other_sexual_orientation', 'physical_disability','psychiatric_or_mental_illness', 'transgender', 'white', 'created_date','publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow','sad', 'likes', 'disagree', 'sexual_explicit','identity_annotator_count', 'toxicity_annotator_count','negative_comment_count']
print_neat_bad_comments(unintended_bias_train_processed, extra_labels)

### Pie Chart Comparision

In [ ]:
toxic_combination_matrix = pie_chart_combined_negative_category(toxic_comment_train_processed,cmn_toxic_params)

### Toxic custom weight comparision

In [ ]:
toxic_weight_comparision(toxic_combination_matrix, cmn_toxic_params)

### Peek into a few examples - comment_text

In [ ]:
'''
Removing Unnecessary characters
'''
def print_textprocessing_difference(df):
    pd.set_option('display.max_columns',None)  
    pd.set_option('display.expand_frame_repr', False)
    pd.set_option('max_colwidth', -1)

    df.loc[df['toxic']==1,['id','comment_text']][:3]
    print(df.iloc[12,1])
    print('\n -------- \n')
    print(text_preprocessing(df.iloc[12,1]))
    
print_textprocessing_difference(toxic_comment_train_processed)

In [ ]:
'''
Removing Stop Words
'''
#toxic_comment_train_processed['comment_text'] = toxic_comment_train_processed['comment_text'].apply(lambda x: without_stopwords(x))
#ubtp = unintended_bias_train_processed[['comment_text','toxic']]
#ubtp['comment_text'] = ubtp['comment_text'].apply(lambda x: without_stopwords(x))
#ubtp = convert_continious_to_discrete_labels(ubtp,['toxic'])

In [ ]:
# xubtp = ubtp.drop(columns=[i for i in ubtp.columns if i not in  ['comment_text','toxic']])
# xubtp['toxic'] = xubtp['toxic'].astype(np.uint8)
# xubtp.memory_usage().sum()
# xtctp = toxic_comment_train_processed.drop(columns=[i for i in toxic_comment_train_processed.columns if i not in  ['comment_text','toxic']])
# xtctp['toxic'] = xtctp['toxic'].astype(np.uint8)
# xtctp.memory_usage().sum()
# train = pd.concat([xtctp[['comment_text','toxic']],xubtp[['comment_text','toxic']]],axis=0)

In [ ]:

#complete_set = int((toxic_comment_train_processed.shape[0] - (toxic_comment_train_processed.shape[0]%8))/8)
#train = train.loc[:12000,:]
#train['comment_text'] = train['comment_text'].apply(lambda x:len(str(x).split())).max()
xtrain = toxic_comment_train_processed.iloc[:12000,1]
ytrain = toxic_comment_train_processed.iloc[:12000,2]
xvalid = validation_processed.iloc[:4000,1]
yvalid = validation_processed.iloc[:4000,3]
xtrain, xvalid, ytrain, yvalid = train_test_split(X,y,stratify=y,random_state=42,train_size=100000,test_size=50000, shuffle=True)

In [ ]:
#tokenizer
max_len = 1500
def regular_tokenizer(xtrain, xvalid):
    token = Tokenizer(num_words=None)
    token.fit_on_texts(list(xtrain) + list(xvalid))
    xtrain_seq = token.texts_to_sequences(xtrain)
    xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
    word_index = token.word_index
    print('Total number of unique words : {}'.format(len(token.word_index)))
    
    return xtrain_seq, xtrain_pad, word_index

xtrain_seq, xtrain_pad, word_index = regular_tokenizer(xtrain, xvalid)

In [ ]:
# xvalid_seq = token.texts_to_sequences(xvalid)
# xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

### TPU + Batchsize

In [ ]:
tf.keras.backend.clear_session()
BATCH_SIZE_PER_REPLICA = 60
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
'''
we are passing 12000*0.8 -> 9600 data points in the train set if batchsize is 64 then 9600/64 = 150 (8*8)
Generally 8 replicas are available so, the provided batch size is split into 8 and processed parallely
Similarly 9600/(12*8) -> 100 batches,

Which is why batch size per replica being 64 throws an error. 9600/(64*8) = 18.75!! 
It can be 60
9600/(60*8) = 20
It can also be 120
9600/(120*8) = 10
'''

### Glove Embeddings

In [ ]:
def extract_glove_embedding_matrix(input_file, word_index):
    '''
    !wget http://nlp.stanford.edu/data/glove.6B.zip
    !unzip -q glove.6B.zip
    '''
    embeddings_index = {}
    f = open('../input/glove-embeddings/glove.6B.100d.txt','r',encoding='utf-8')
    '''
    tqdm - is just used to display a progress bar while the loop is in progress
    The following for loop is used to fetch the values of the weights stored in the pre-trained model 'f' into the embeddings_index
    '''
    for line in tqdm(f):
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray([float(val) for val in values[1:]])
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))
    '''
    Here we initialize the embedding matrix with the length of our unique word list that is the word_index and 100 columns. 
    100 because we extracted the weights of glove.6B.100d.txt. it would be 300 if we use glove.6B.300d.txt
    The list of all the pretrained glove model can be found it this link -> https://nlp.stanford.edu/projects/glove/

    we fill the embedding_matrix with only our unique words under analysis and not all the 4,00,000 word vectors.
    '''
    embedding_matrix = np.zeros((len(word_index) + 1, 100))
    for word, i in tqdm(word_index.items()):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix
            
input_file = '../input/glove-embeddings/glove.6B.100d.txt'
embedding_matrix = extract_glove_embedding_matrix(input_file, word_index)

### SimpleRNN

In [ ]:
def execute_simplernn():
    with strategy.scope():

        model = Sequential()
        model.add(Embedding(len(word_index) + 1,100,weights=[embedding_matrix],input_length=max_len))
        model.add(SimpleRNN(128))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.summary()
    history = model.fit(xtrain_pad, ytrain, epochs=5, batch_size=GLOBAL_BATCH_SIZE, verbose =1)
    scores = model.predict(xvalid_pad)
    print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))
    do_history_stuff(history)
tf.keras.backend.clear_session()
execute_simplernn()

### LSTM

In [ ]:
%%time
def execute_lstm():
    with strategy.scope():
        lstm_model = Sequential()
        '''
        The embedding layer is the input layer and we know that each word has been modified into a vector of length 100
        weights - glove's weights for the unique words in our analysis
        maximum length of each comment is 1500 which we defined earlier itself
        '''
        lstm_model.add(Embedding(len(word_index) + 1,100, weights=[embedding_matrix],input_length=max_len,trainable=False))
        lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
        lstm_model.add(Dense(1, activation='sigmoid'))
        lstm_model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy']) 

    lstm_model.summary()
    lstm_history = lstm_model.fit(xtrain_pad, ytrain, epochs=10, batch_size=GLOBAL_BATCH_SIZE, verbose=1)
    lstm_scores = lstm_model.predict(xvalid_pad)
    print("Auc: %.2f%%" % (roc_auc(lstm_scores,yvalid)))
    do_history_stuff(lstm_history)
tf.keras.backend.clear_session()
execute_lstm()

### GRU

In [ ]:
'''
GRU Model
'''
def execute_gru():
    with strategy.scope():
         gru_model = Sequential()
         gru_model.add(Embedding(len(word_index) + 1,100,weights=[embedding_matrix],input_length=max_len,trainable=False))
         gru_model.add(SpatialDropout1D(0.3))
         gru_model.add(GRU(128))
         gru_model.add(Dense(1, activation='sigmoid'))
         gru_model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   

    gru_model.summary()
    gru_history = gru_model.fit(xtrain_pad, ytrain, epochs=30, batch_size=GLOBAL_BATCH_SIZE, verbose=1)
    gru_scores = gru_model.predict(xvalid_pad)
    print("Auc: %.2f%%" % (roc_auc(gru_scores,yvalid)))
    do_history_stuff(gru_history)
tf.keras.backend.clear_session()
execute_gru()

### Bi-directional RNN

In [ ]:
'''
Bi-Directional RNN
'''
def execute_bidirectionalrnn():
    with strategy.scope():
        bidirectional_model = Sequential()
        bidirectional_model.add(Embedding(len(word_index) + 1,100,weights=[embedding_matrix],input_length=max_len,trainable=False))
        bidirectional_model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
        bidirectional_model.add(Dense(1,activation='sigmoid'))
        bidirectional_model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

    bidirectional_model.summary()
    bidirectional_history = bidirectional_model.fit(xtrain_pad, ytrain, epochs=5, batch_size=GLOBAL_BATCH_SIZE, verbose=1)
    bidirectional_scores = bidirectional_model.predict(xvalid_pad)
    print("Auc: %.2f%%" % (roc_auc(bidirectional_scores,yvalid)))
    do_history_stuff(bidirectional_history)
tf.keras.backend.clear_session() 
execute_bidirectionalrnn()

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=192):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen) #max_length=maxlen
    tokenizer.enable_padding() #max_length=maxlen
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

def bert_encode(texts, tokenizer, max_len=512): 
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
'''
Load Data for bert
'''
train = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [ ]:
'''
Defining Tokenizer
'''
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
tokenizer.save_pretrained('.')
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
'''
Perform Encoding
'''
x_train = fast_encode(train.comment_text.astype(str), fast_tokenizer, maxlen=192)
x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=192)
y_train = train.toxic.values
y_valid = valid.toxic.values


'''Data Generator'''
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = GLOBAL_BATCH_SIZE

In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))

valid_dataset = (tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(BATCH_SIZE).cache().prefetch(AUTO))


In [ ]:
tf.keras.backend.clear_session()

def build_model(transformer, loss='binary_crossentropy', max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.35)(cls_token)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=3e-5), loss=loss, metrics=[tf.keras.metrics.AUC()])
    
    return model

from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=.2):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed


with strategy.scope():
    transformer_layer = transformers.TFBertModel.from_pretrained('bert-base-uncased')
    model = build_model(transformer_layer, loss=focal_loss(gamma=1.5), max_len=192)
model.summary()

In [ ]:
n_steps = xtrain.shape[0] // GLOBAL_BATCH_SIZE
bert_history = model.fit(train_dataset, epochs = 3, batch_size = GLOBAL_BATCH_SIZE, steps_per_epoch = n_steps)

In [ ]:
'''
encoding on test to predict and create submission file
'''
x_test = fast_encode(test.content.astype(str), fast_tokenizer,maxlen=192)

test_dataset = (tf.data.Dataset.from_tensor_slices(x_test).batch(BATCH_SIZE))

prediction = model.predict(test_dataset)

In [ ]:
len(prediction)

In [ ]:
# train_dataset=(tf.data.Dataset
#               .from_tensor_slices((x_train_d,y_train_d))
#               .repeat()
#               .shuffle(2048)
#               .batch(BATCH_SIZE)
#               .cache()
#               .prefetch(AUTO))

In [ ]:
# these are the different token attributes    
  
#  TEXT	LEMMA	POS	TAG	DEP	SHAPE	ALPHA	STOP
details = {'text': [],
           'lemma': [],
           'pos' : [],
           'tag' : [],
           'dep' : [],
           'shape' : [],
           'alpha' : [],
           'stop' : []
          }
details_id_map = {}

for id, comment in zip(train['id'],train['comment_text']):
    details = {'text': [],
           'lemma': [],
           'pos' : [],
           'tag' : [],
           'dep' : [],
           'shape' : [],
           'alpha' : [],
           'stop' : []
          }
    for token in comment:
        details['text'].append(token.text)
        details['lemma'].append(token.lemma_)
        details['pos'].append(token.pos_)
        details['tag'].append(token.tag_)
        details['dep'].append(token.dep_)
        details['shape'].append(token.shape_)
        details['alpha'].append(token.is_alpha)
        details['stop'].append(token.is_stop)
    
    details_id_map[id] = details

In [ ]:
nlp(train['comment_text'][0])

In [ ]:
# from wordcloud import WordCloud,STOPWORDS
# wordcloud = WordCloud(stopwords=STOPWORDS,
#                       background_color='black',
#                       width=3000,
#                       height=2500
#                      ).generate(commonWord)
# plt.figure(1,figsize=(12, 12))
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.show()